In [87]:
import pandas as pd
import numpy as np
import torch
!pip install fastai==0.7.0
!pip install torchtext==0.2.3

import fastai
from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *
from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *
from fastai.text import *

import dill as pickle
!pip install spacy
!python -m spacy download en
import spacy
import html

import pickle
import collections
import math


    Linking successful
    /Users/michaelschimpke/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/en_core_web_sm
    -->
    /Users/michaelschimpke/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [793]:
text = 'Hello what is your'
labels = 0
text = pd.Series(text)
labels = pd.Series(labels)

text = pd.Series.as_matrix(text)
labels = pd.Series.as_matrix(labels)

colNames = ['labels','text']
textdf = pd.DataFrame({'text':text, 'labels':labels}, columns = colNames)
textdf.to_csv('Test_lm/text.csv', header=False, index=False)

/Users/michaelschimpke/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/michaelschimpke/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [794]:
re1 = re.compile(r'  +')

# cleaning function
def cleaning(x):
   x = x.replace('#', '').replace('&amp;', '&')
   #return re1.sub(' ', html.unescape(x))
   return re1.sub(' ', re.sub('https?://[A-Za-z0-9./]+', '',html.unescape(x)))


# tokenizer function
def get_texts(df, n_lbls=1):
    labels = np.unique(df.iloc[:,range(n_lbls)].values, return_inverse=True)[n_lbls]
    texts = f'\n{BOS} ' + df[n_lbls].astype(str)
    for i in range(n_lbls+1, len(df.columns)): 
        texts += df[i].astype(str)
    texts = texts.apply(cleaning).values.astype(str)
    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    return tok, list(labels)

# iterator function
def get_all(df, n_lbls):
    tok, labels = [], []
    for i, r in enumerate(df):
        print(i)
        tok_, labels_ = get_texts(r, n_lbls)
        tok += tok_;
        labels += labels_
    return tok, labels

In [795]:
chunksize = 1
BOS = 'xbos'
textdf = pd.read_csv('Test_lm/text.csv', header=None, chunksize=chunksize)

tok_text = get_all(textdf,1)
tok = (tok_text[0])
print(tok)

0
[['\n', 'xbos', 'hello', 'what', 'is', 'your']]


In [7]:
with open('lm/itos.pkl', 'rb') as pickle_file:
    itos = pickle.load(pickle_file)

stoi = collections.defaultdict(lambda:0, 
                               {v:k for k,v in enumerate(itos)})

list(stoi.items())[0:10]

[('_unk_', 0),
 ('_pad_', 1),
 ('\n', 2),
 ('xbos', 3),
 ('.', 4),
 ('to', 5),
 ('i', 6),
 ('the', 7),
 ('t_up', 8),
 ('!', 9)]

In [796]:
textLm = [[stoi[o] for o in p] for p in tok]
textLm = textLm[0]
print(textLm)

[2, 3, 874, 61, 22, 31]


In [797]:
def oneHot(sequences, dimension=len(itos)):
      results = np.zeros((len(sequences), dimension))
      for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
      return results

onehot = oneHot(textLm)
one = np.array([onehot])

np.set_printoptions(edgeitems=4)
print(one[0])
onehot.shape

[[0. 0. 1. 0. ... 0. 0. 0. 0.]
 [0. 0. 0. 1. ... 0. 0. 0. 0.]
 [0. 0. 0. 0. ... 0. 0. 0. 0.]
 [0. 0. 0. 0. ... 0. 0. 0. 0.]
 [0. 0. 0. 0. ... 0. 0. 0. 0.]
 [0. 0. 0. 0. ... 0. 0. 0. 0.]]


(6, 4409)

In [798]:
lm_wgts_unfr = torch.load('models/lm2.h5', map_location=lambda storage, 
                  loc: storage)

embed = np.matmul(one[0], lm_wgts_unfr['0.encoder.weight'])
print(embed)
embed.shape

[[-0.37896 -0.37842 -0.19198  0.09403 ... -0.21748  0.04106  0.24902  0.09481]
 [-0.11337 -0.29701 -0.09582  0.09519 ...  0.00417 -0.00866  1.10079 -0.14191]
 [-0.16737 -0.44848 -0.04136  0.17024 ... -0.02828  0.05867  0.33854  0.18265]
 [ 0.04131  0.61636 -0.12456  0.37107 ...  0.05903  0.04707  0.64659  0.08474]
 [-0.1449   0.9734  -0.10638 -0.1773  ...  0.01902 -0.06392 -0.54776  0.10417]
 [-0.23652 -0.98352 -0.09888 -0.09476 ...  0.17836 -0.10366  0.97914  0.2162 ]]


(6, 400)

## LSTM Tryout

In [15]:
type(embed)

numpy.ndarray

In [468]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def LSTM (input, model, nh, stage):
    #input weights and bias
    wii = np.matrix(model['0.rnns.'+stage+'.module.weight_ih_l0'][:nh].numpy())
    wif = np.matrix(model['0.rnns.'+stage+'.module.weight_ih_l0'][nh:2*nh].numpy())
    wig = np.matrix(model['0.rnns.'+stage+'.module.weight_ih_l0'][2*nh:3*nh].numpy())
    wio = np.matrix(model['0.rnns.'+stage+'.module.weight_ih_l0'][3*nh:4*nh].numpy())
    
    bii = np.matrix(model['0.rnns.'+stage+'.module.bias_ih_l0'][:nh].numpy())
    bif = np.matrix(model['0.rnns.'+stage+'.module.bias_ih_l0'][nh:2*nh].numpy())
    big = np.matrix(model['0.rnns.'+stage+'.module.bias_ih_l0'][2*nh:3*nh].numpy())
    bio = np.matrix(model['0.rnns.'+stage+'.module.bias_ih_l0'][3*nh:4*nh].numpy())
    #output weights and bias
    whi = np.matrix(model['0.rnns.'+stage+'.module.weight_hh_l0_raw'][:nh].numpy())
    whf = np.matrix(model['0.rnns.'+stage+'.module.weight_hh_l0_raw'][nh:2*nh].numpy())
    whg = np.matrix(model['0.rnns.'+stage+'.module.weight_hh_l0_raw'][2*nh:3*nh].numpy())
    who = np.matrix(model['0.rnns.'+stage+'.module.weight_hh_l0_raw'][3*nh:4*nh].numpy())
    
    bhi = np.matrix(model['0.rnns.'+stage+'.module.bias_hh_l0'][:nh].numpy())
    bhf = np.matrix(model['0.rnns.'+stage+'.module.bias_hh_l0'][nh:2*nh].numpy())
    bhg = np.matrix(model['0.rnns.'+stage+'.module.bias_hh_l0'][2*nh:3*nh].numpy())
    bho = np.matrix(model['0.rnns.'+stage+'.module.bias_hh_l0'][3*nh:4*nh].numpy())
    
    ht = np.zeros(nh)
    ct = np.zeros(nh)
    hidden_matrix = np.empty((0,nh))
    
    for t,v in enumerate(input):
        it = sigmoid(np.matmul(wii,input[t]) + bii + np.matmul(ht,whi) + bhi)
        ft = sigmoid(np.matmul(wif,input[t]) + bif + np.matmul(ht,whf) + bhf)
        gt = np.tanh(np.matmul(wig,input[t]) + big + np.matmul(ht,whg) + bhg)
        ot = sigmoid(np.matmul(wio,input[t]) + bio + np.matmul(ht,who) + bho)
        ct = np.multiply(ft,ct) + np.multiply(it,gt)
        ht = np.multiply(ot,np.tanh(ct))
        hidden_matrix = np.append(hidden_matrix, ht, axis=0)
    
    hidden_state = np.array(hidden_matrix)
    return hidden_state

In [467]:
np.matrix(lm_wgts_unfr['0.rnns.0.module.weight_hh_l0_raw'][:nh, nh:2*nh].numpy())

ValueError: result of slicing is an empty tensor

In [799]:
alpha = LSTM(embed, lm_wgts_unfr, 1150, stage = '0')
beta = LSTM(alpha, lm_wgts_unfr, 1150, stage = '1')
gamma = LSTM(beta, lm_wgts_unfr, 400, stage = '2')

In [800]:
print(alpha)
alpha.shape

[[-0.15425  0.00635  0.06167  0.00478 ...  0.00305  0.12759 -0.05522  0.03215]
 [ 0.10493 -0.01119  0.06614  0.42933 ...  0.01206  0.6021  -0.17489  0.07547]
 [-0.21283  0.04279 -0.09859  0.10598 ...  0.00062  0.5574  -0.0191   0.01187]
 [-0.02336  0.04026 -0.02976  0.01746 ... -0.32244  0.02244 -0.0061  -0.05338]
 [-0.00781  0.0301   0.28934 -0.02776 ...  0.00196 -0.00132 -0.01145  0.00249]
 [-0.01336  0.2432  -0.05934  0.04848 ... -0.18229  0.04554  0.00846  0.07499]]


(6, 1150)

In [801]:
print(beta)
beta.shape

[[ 0.0062   0.00442 -0.11493 -0.01229 ...  0.04158 -0.00033 -0.0592  -0.00177]
 [ 0.00318  0.00068 -0.07347  0.00099 ...  0.01347  0.01952 -0.20328 -0.01462]
 [ 0.01166  0.00504 -0.08449 -0.00989 ... -0.00054  0.05532 -0.0907  -0.00557]
 [ 0.35734  0.01474 -0.01913 -0.02387 ...  0.00054  0.06855 -0.44543 -0.00195]
 [ 0.234   -0.03623 -0.00133 -0.00509 ... -0.00262  0.2074  -0.0772  -0.00466]
 [ 0.0707   0.00277  0.02339 -0.00714 ... -0.06681  0.10778 -0.13495 -0.02039]]


(6, 1150)

In [802]:
print(gamma)
gamma.shape

[[-0.07604  0.21237 -0.61837 -0.01263 ... -0.00011 -0.09578  0.14449  0.0666 ]
 [-0.29052  0.44175 -0.76797 -0.11513 ...  0.05797 -0.33318  0.06172 -0.18974]
 [-0.21353  0.20345 -0.68511 -0.02195 ... -0.17163 -0.35223 -0.06984  0.06681]
 [-0.04537  0.22954 -0.61536 -0.01184 ... -0.09889 -0.41603 -0.06546  0.23309]
 [ 0.0198   0.17629 -0.66869 -0.01044 ... -0.17377 -0.13723  0.04795  0.37522]
 [-0.14241 -0.039   -0.51813 -0.14967 ... -0.51524 -0.31871  0.08614  0.00981]]


(6, 400)

In [803]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

prediction = np.matmul(gamma[5], np.transpose(lm_wgts_unfr['1.decoder.weight']))
pred = softmax(prediction)

In [618]:
print(pred[4400:4413])

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [603]:
np.where(pred==max(pred))
#max(pred)

(array([55]),)

In [805]:
pred_tok = pred.argsort()[-19:][::-1]
pred_prob = pred[pred_tok]
print(pred_tok)

[329 230 472 454 274 118   0  55 385 228 316 325 437  34  52  66 481 791 162]


In [804]:
print(pred_tok)

[145   7 149 512 221 139   0 306 192 313  34 235  50 467 890  70 375  92 316]


### Prediction

In [806]:
vs = 4409
stoi_df = pd.DataFrame(list(stoi.keys())[:vs], list(stoi.values())[:vs], columns = ['word'])
stoi_df['probability'] = list(pred)
print(stoi_df.iloc[pred_tok])

         word  probability
329      team     0.037297
230      best     0.025882
472    policy     0.022328
454   company     0.021925
274   website     0.016830
118   airline     0.014698
0       _unk_     0.013337
55          "     0.012549
385      site     0.012248
228     staff     0.011937
316   awesome     0.010509
325  business     0.009440
437      show     0.009026
34        not     0.008619
52    service     0.008091
66   customer     0.007573
481   account     0.007449
791      kind     0.007408
162       new     0.007303


### Alternativer Weg

In [780]:
stoi_df = pd.DataFrame(stoi, index = stoi.values())
stoi_df = pd.DataFrame(stoi_df.columns[:vs], stoi_df.iloc[0][:vs])
stoi_df['prob'] = np.array(pred)
print(stoi_df.iloc[pred_tok])

           0      prob
0                     
7        the  0.534758
11         a  0.092828
8       t_up  0.055385
42         -  0.054523
19        my  0.050886
15    flight  0.026186
31      your  0.016935
40      this  0.013233
0      _unk_  0.013048
69       all  0.009171
165  another  0.007184
54        an  0.006042
90       one  0.005903
504     both  0.005573
60         2  0.005407
30      that  0.004406
55         "  0.004212
346    board  0.003879
74      hold  0.003347


# Schneller Durchlauf der Prediction

Dafür muss allerdings der Hauptcode schon ausgeführt worden sein

In [762]:
#Preparation
text = 'my flight was on'
labels = 0
text = pd.Series(text)
labels = pd.Series(labels)

text = pd.Series.as_matrix(text)
labels = pd.Series.as_matrix(labels)

colNames = ['labels','text']
textdf = pd.DataFrame({'text':text, 'labels':labels}, columns = colNames)
textdf.to_csv('Test_lm/text.csv', header=False, index=False)

/Users/michaelschimpke/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
/Users/michaelschimpke/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [763]:
chunksize = 1
BOS = 'xbos'
textdf = pd.read_csv('Test_lm/text.csv', header=None, chunksize=chunksize)

#Tokenizer
tok_text = get_all(textdf,1)
tok = (tok_text[0])

textLm = [[stoi[o] for o in p] for p in tok]
textLm = textLm[0]

#One-Hot Encoding und Embedding Layer
onehot = oneHot(textLm)
one = np.array([onehot])
embed = np.matmul(one[0], lm_wgts_unfr['0.encoder.weight'])

#LSTMs mit Hidden States
alpha = LSTM(embed, lm_wgts_unfr, 1150, stage = '0')
beta = LSTM(alpha, lm_wgts_unfr, 1150, stage = '1')
gamma = LSTM(beta, lm_wgts_unfr, 400, stage = '2')

In [764]:
#Softmax Layer
prediction = np.matmul(gamma[-1], np.transpose(lm_wgts_unfr['1.decoder.weight']))
pred = softmax(prediction)

In [754]:
#Prediction
pred_tok = pred.argsort()[-19:][::-1]
print(stoi_df.iloc[pred_tok])

        word  probability
13         ,     0.031965
4          .     0.041592
38        at     0.000056
18       and     0.004235
9          !     0.162758
23        in     0.000078
14       for     0.000424
6          i     0.003539
19        my     0.011067
35      with     0.000172
286    night     0.000010
15    flight     0.005140
17        on     0.000064
57       ...     0.017402
7        the     0.002102
251  morning     0.000070
123        (     0.000170
42         -     0.000765
44       but     0.004279


In [702]:
onehot

array([[0., 0., 1., 0., ..., 0., 0., 0., 0.],
       [0., 0., 0., 1., ..., 0., 0., 0., 0.],
       [0., 0., 0., 0., ..., 0., 0., 0., 0.]])

In [743]:
print(pred_tok.sort(axis=0))

None


# Test- und Probierzeilen

In [199]:
nh = 1150
ht = np.zeros(nh)
lm_wgts_unfr['0.rnns.0.module.weight_ih_l0'][nh:2*nh].size()

torch.Size([1150, 400])

In [296]:
a = np.matrix(lm_wgts_unfr['0.rnns.0.module.weight_hh_l0_raw'][:nh].numpy())
b = np.zeros(nh)
c = sigmoid(np.matmul(a,b))
c.shape

(1, 1150)

In [269]:
b = np.matmul(a, embed[0])
b.shape
c = np.add(b,lm_wgts_unfr['0.rnns.0.module.bias_ih_l0'][nh:2*nh])
c

matrix([[ 0.3324 , -3.77108, -0.25524, -4.84101, ..., -1.42723, -4.43494, -2.45823, -3.24682]])

In [184]:
np.matmul(lm_wgts_unfr['0.rnns.0.module.weight_ih_l0'][:1150],embed[0])

array([ 0.68714, -3.42605,  0.19375, -4.35618, ..., -1.10531, -3.83029, -2.00408, -2.96782])

In [185]:
embed[1].shape
test1 = np.matmul(lm_wgts_unfr['0.rnns.0.module.weight_ih_l0'][:1150],embed[1])
test1

array([ 2.0036 , -0.51683,  1.0608 ,  1.59737, ..., -0.00028, -0.4793 ,  0.06419, -0.0609 ])

In [186]:
test2 = np.matmul(lm_wgts_unfr['0.rnns.0.module.weight_ih_l0'][:1150],embed[1])
test2

array([ 2.0036 , -0.51683,  1.0608 ,  1.59737, ..., -0.00028, -0.4793 ,  0.06419, -0.0609 ])

In [187]:
np.multiply(test1, test2)

array([4.01441, 0.26712, 1.12531, 2.55158, ..., 0.     , 0.22973, 0.00412, 0.00371])

In [99]:
test3 = lm_wgts_unfr['0.rnns.0.module.weight_ih_l0'][:1150]*embed[0]
print(test3)

TypeError: mul received an invalid combination of arguments - got (numpy.ndarray), but expected one of:
 * (float value)
      didn't match because some of the arguments have invalid types: ([31;1mnumpy.ndarray[0m)
 * (torch.FloatTensor other)
      didn't match because some of the arguments have invalid types: ([31;1mnumpy.ndarray[0m)


In [306]:
for t,values in enumerate(embed):
       print(t)

0
1
2
3
4
5


In [319]:
wii = np.matrix(lm_wgts_unfr['0.rnns.0.module.weight_ih_l0'][:nh].numpy())
whi = np.matrix(lm_wgts_unfr['0.rnns.0.module.weight_hh_l0_raw'][:nh].numpy())
bii = np.matrix(lm_wgts_unfr['0.rnns.0.module.bias_ih_l0'][:nh].numpy())
bhi = np.matrix(lm_wgts_unfr['0.rnns.0.module.bias_hh_l0'][:nh].numpy())

for t, values in enumerate(embed):
        it = sigmoid(np.matmul(wii,embed[t]) + bii + np.matmul(whi,np.transpose(np.zeros(nh))) + bhi)
        
it.shape

(1, 1150)

In [114]:
#hadamard product
np.multiply(np.ones(1150), np.zeros(1150))

array([0., 0., 0., 0., ..., 0., 0., 0., 0.])

In [115]:
x1 = np.arange(9.0).reshape((3, 3))
x2 = np.arange(3.0)
np.multiply(x1, x2)

array([[ 0.,  1.,  4.],
       [ 0.,  4., 10.],
       [ 0.,  7., 16.]])

array([[0., 1., 2.],
       [3., 4., 5.],
       [6., 7., 8.]])

In [117]:
x2

array([0., 1., 2.])